In [107]:
#getting our own spotify data

In [108]:
#importing libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
import skimage


In [109]:
#Authentication - without user using Spotify's Web API and making a new spotify developer account 
client_credentials_manager = SpotifyClientCredentials(
    client_id="14ab08163a0649be865dc476a3076fd7", client_secret="0ce7157e93604e008992ec4db58d8e70")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [110]:
#Getting user playlist
playlist_link = "https://open.spotify.com/playlist/5szG1jvEJ6KBLjvklDrdtt?si=48eb7c244c754311"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"]
              for x in sp.playlist_tracks(playlist_URI)["items"]]


In [111]:
#Acosticness confidence measure from 0.0 to 1.0 of whether the track is acoustic.
#Danceability measure describes how suitable a track is for dancing.
#duration_ms is the duration of the song track in milliseconds.
#Energy represents a perceptual measure of intensity and activity.
#Instrumentalness predicts whether a track contains vocals or not.
#Loudness of a track in decibels(dB).
#Liveness detects the presence of an audience in the recording.
#Speechiness detects the presence of spoken words in a track
#Time_signature is an estimated overall time signature of a track.
#Key the track is in. Integers map to pitches using standard Pitch Class notation.
#Valence measures from 0.0 to 1.0 describing the musical positiveness conveyed by a track.
#Target value describes the encoded value of 0 and 1. 0 means listener has not saved the song and 1 means listener have saved the song.
#Tempo is in beats per minute(BPM).
#Mode indicates the modality(major or minor) of the song.
#Song_title is the name of the song.
#Artist is the singer of the song.
#URL is the link to the song.



In [112]:
#making a new csv file with th data and headers as the features of the audio features and also Song Name and Artist Name
song_name = []
artist_name = []
acousticness = []
danceability = []
duration_ms = []
energy = []
instrumentalness = []
loudness = []
liveness=[]
speechiness = []
time_signature = []
key = []
valence = []
tempo = []
mode = []
target = []
url = []
#get track name and artist name
for track_uri in track_uris:
    track = sp.track(track_uri)
    song_name.append(track["name"])
    artist_name.append(track["artists"][0]["name"])
    audio_features = sp.audio_features(track_uri)
    acousticness.append(audio_features[0]["acousticness"])
    danceability.append(audio_features[0]["danceability"])
    duration_ms.append(audio_features[0]["duration_ms"])
    energy.append(audio_features[0]["energy"])
    instrumentalness.append(audio_features[0]["instrumentalness"])
    loudness.append(audio_features[0]["loudness"])
    liveness.append(audio_features[0]["liveness"])
    speechiness.append(audio_features[0]["speechiness"])
    time_signature.append(audio_features[0]["time_signature"])
    key.append(audio_features[0]["key"])
    valence.append(audio_features[0]["valence"])
    tempo.append(audio_features[0]["tempo"])
    mode.append(audio_features[0]["mode"])
    target.append(1)
    url.append(track["external_urls"]["spotify"])
#make a csv file with the data
df = pd.DataFrame({'Song Name':song_name, 'Artist Name':artist_name, 'Acousticness':acousticness, 'Danceability':danceability, 'Duration_ms':duration_ms, 'Energy':energy, 'Instrumentalness':instrumentalness, 'Loudness':loudness,'Liveness':liveness ,'Speechiness':speechiness, 'Time_signature':time_signature, 'Key':key, 'Valence':valence, 'Tempo':tempo, 'Mode':mode, 'Target':target, 'URL':url})
df.to_csv('audio_features.csv', index=False)


In [113]:
df = pd.read_csv("Audio_features.csv")
df.head()

,Song Name,Artist Name,Acousticness,Danceability,Duration_ms,Energy,Instrumentalness,Loudness,Liveness,Speechiness,Time_signature,Key,Valence,Tempo,Mode,Target
0,Hey Joe,Jimi Hendrix,0.00603,0.346,210160,0.768,0.380000,-5.695,0.0244,0.0377,4,9,0.532,169.492,1,1
1,Purple Haze,Jimi Hendrix,0.00876,0.533,170813,0.905,0.578000,-5.270,0.0698,0.0754,4,2,0.486,108.900,1,1
2,Hotel California - 2013 Remaster,Eagles,0.00574,0.579,391376,0.508,0.000494,-9.484,0.0575,0.0270,4,2,0.609,147.125,1,1
3,Stairway to Heaven - Remaster,Led Zeppelin,0.58000,0.338,482830,0.340,0.003200,-12.049,0.1160,0.0339,4,9,0.197,82.433,0,1
4,Sweet Child O' Mine,Guns N' Roses,0.08660,0.454,354520,0.910,0.099600,-7.766,0.1160,0.0448,4,6,0.629,125.116,1,1


In [114]:
from sklearn.preprocessing import MinMaxScaler
feature_cols = ['Acousticness', 'Danceability', 'Duration_ms', 'Energy',
                'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode',
                'Speechiness', 'Tempo', 'Time_signature', 'Valence', ]

scaler = MinMaxScaler()
normalized_df = scaler.fit_transform(df[feature_cols])

print(normalized_df[:2])


[[0.00750586 0.18698061 0.19617132 0.76440461 0.42175361 0.81818182
  0.         0.81543087 1.         0.02704678 0.74654768 0.75
  0.51562846]
 [0.01091847 0.44598338 0.12475429 0.93982074 0.64150943 0.18181818
  0.04816465 0.84262043 1.         0.11890838 0.28831581 0.75
  0.46464199]]


In [115]:
# Create a pandas series with song titles as indices and indices as series values
indices = pd.Series(df.index, index=df['Song Name']).drop_duplicates()

# Create cosine similarity matrix based on given matrix
cosine = cosine_similarity(normalized_df)


def generate_recommendation(song_title,limit_songs, model_type=cosine):
    """
    Purpose: Function for song recommendations 
    Inputs: song title and type of similarity model
    Output: Pandas series of recommended songs
    """
    # Get song indices
    index = indices[song_title]
    
    # Get list of songs for given songs
    score = list(enumerate(model_type[indices[song_title]]))
    # Sort the most similar songs
    similarity_score = sorted(score, key=lambda x: x[1], reverse=True)
    # Select the top-user input recommended songs
    similarity_score = similarity_score[1:limit_songs+1]
    top_songs_index = [i[0] for i in similarity_score]
    # Top user-input value recommended songs
    top_songs = df['Song Name'].iloc[top_songs_index]
    top_songs_artists = df['Artist Name'].iloc[top_songs_index]
    result_frame=pd.DataFrame({'Song Name':top_songs, 'Artist Name':top_songs_artists})
    return result_frame
    


In [116]:
#Print all the songs in the playlist
for i in range(len(song_name)):
    print(song_name[i],":",artist_name[i])


Hey Joe : Jimi Hendrix
Purple Haze : Jimi Hendrix
Hotel California - 2013 Remaster : Eagles
Stairway to Heaven - Remaster : Led Zeppelin
Sweet Child O' Mine : Guns N' Roses
Knockin' On Heaven's Door : Guns N' Roses
Anastasia : Slash
Paradise City : Guns N' Roses
Comfortably Numb : Pink Floyd
All Along the Watchtower : Jimi Hendrix
Voodoo Child (Slight Return) : Jimi Hendrix
Little Wing : Jimi Hendrix
Gravity - Live at the Nokia Theatre, Los Angeles, CA - December 2007 : John Mayer
Free Bird : Lynyrd Skynyrd
Do I Wanna Know? : Arctic Monkeys
Sweater Weather : The Neighbourhood
Seven Nation Army : The White Stripes
White Room : Cream
Crossroads - Live : Cream
Let It Be - Remastered 2009 : The Beatles
Bold as Love : John Mayer
Moby Dick - Remaster : Led Zeppelin
Kashmir - Remaster : Led Zeppelin
Whole Lotta Love - Remaster : Led Zeppelin
Immigrant Song - Remaster : Led Zeppelin
Black Dog - Remaster : Led Zeppelin
Heartbreaker - 1990 Remaster : Led Zeppelin
Sultans Of Swing : Dire Straits


In [120]:
song_value=input("Write the song name you want to recommend from this playlist")

In [121]:
number=int(input("Number of Recommended Songs you want to see"))  

In [122]:
print("Recommended Songs:")
print(generate_recommendation(song_value,number,cosine))

Recommended Songs:
                                            Song Name            Artist Name
53                               Something In The Way                Nirvana
31                              No Quarter - Remaster           Led Zeppelin
76                                           Marooned             Pink Floyd
46  Tears in Heaven - Acoustic; Live at MTV Unplug...           Eric Clapton
65                                            Gravity  Jamie Harrison Guitar
67                                             S.R.V.           Eric Johnson
70                                              Polly                Nirvana
21                               Moby Dick - Remaster           Led Zeppelin
78                           Eruption - 2015 Remaster              Van Halen
12  Gravity - Live at the Nokia Theatre, Los Angel...             John Mayer


In [ ]:
#generate images of the recommended songs
# Create a figure with 6 rows and 2 columns
# fig, ax = plt.subplots(6, 2, figsize=(10, 20))
# # Plot the top 10 songs
# for i,